In [26]:
import pandas as pd
import requests
from time import sleep
from tqdm import tqdm
from datetime import datetime
from core.config import Config, get_data
from core.proposicao import Proposicao, ArvoreApensados, Node, ProposicaoAutor, ProposicaoDetalhes
from core.votacao import Votacao, Voto
import matplotlib.pyplot as plt

In [3]:
def request( url) -> list[dict]:
    arqv = []
    flag = True
    while flag:
        try:
            response = requests.get(url)
        except TimeoutError:
            print("Timeout: sleeping for 60 secs")
            sleep(60)
            continue
        except requests.exceptions.ConnectTimeout:
            print("Timeout: sleeping for 60 secs")
            sleep(60)
            continue
        except requests.exceptions.Timeout:
            print("Timeout: sleeping for 60 secs")
            sleep(60)
            continue
        except requests.exceptions.ConnectionError:
            print("ConnectionError: sleeping for 60 secs")
            sleep(60)
            continue
        if response.status_code == 200:
            resp = response.json()
            if isinstance(resp["dados"], list):
                arqv += resp["dados"]
            else:
                arqv.append(resp["dados"])
            rels = [link["rel"] for link in resp["links"]]
            if "next" in rels:
                url = resp["links"][rels.index("next")]["href"]
            else:
                flag = False
        else:
            raise ConnectionError(response.status_code)
    return arqv

def get_arvore_apensados( list_props: list[Proposicao]) -> ArvoreApensados:
    def find_related_projs(prop: Proposicao):
        config = Config(endpoint=f'proposicoes/{prop.id_prop}/relacionadas')
        resp = get_data(config.url)

        proposicoes_relacionadas = []
        for item in resp:
            url_detalhes = item['uri']
            detalhes = request(url_detalhes)[0]
            proposicoes_relacionadas.append(
                Proposicao(
                    id_prop=str(detalhes['id']),
                    sigla_tipo=detalhes['siglaTipo'],
                    ano=detalhes['ano'],
                    ementa=detalhes['ementa'],
                    numero=detalhes['numero'],
                    is_prl=detalhes['siglaTipo'] == 'PRL',
                    praca='cd',
                    data_apresentacao=datetime.strptime(
                        detalhes['dataApresentacao'], "%Y-%m-%dT%H:%M")
                )
            )

        return proposicoes_relacionadas

    # percorrer árvore de apensados
    def montar_arvore(props_ref: list[Proposicao], p_bar: tqdm):
        final = []
        for prop in props_ref:
            resp = find_related_projs(prop)
            node = Node(prop, resp)
            if len(resp) > 0:
                p_bar.total += len(resp)
                p_bar.refresh()
                children = montar_arvore(resp, p_bar)
                node.children = children
            p_bar.update(1)
            final.append(node)
        return final

    pbar = tqdm(list_props, desc='Árvore de Apensados')
    arvore = montar_arvore(list_props, pbar)
    pbar.close()
    return ArvoreApensados(arvore)

def find_projects_by_name(proposicoes: list[Proposicao]) -> list[Proposicao]:
    "Find projects by name."
    # encontrar ids dos projetos
    for proposicao in proposicoes:
        sigla_tipo = proposicao.sigla_tipo
        numero = proposicao.numero
        ano = proposicao.ano
        config = Config(endpoint='proposicoes', parameters={'siglaTipo': sigla_tipo, 'numero': numero, 'ano': ano})
        data = get_data(config.url)
        if len(data) > 0:
            proposicao.id_prop = str(data[0]["id"])
        else:
            print(f"{sigla_tipo} {numero}/{ano} não encontrado\n URL: {config.url}")
    return proposicoes

def get_autores(list_projs: list[Proposicao]) -> list[ProposicaoAutor]:
    # pegar autores
    autores_relacionadas = []
    for proj in tqdm(list_projs, desc="Autores"):
        config = Config(endpoint=f"proposicoes/{proj.id_prop}/autores")
        autores = get_data(config.url)
        resp = []
        for autor in autores:
            autor_obj = ProposicaoAutor(
                id_prop=str(proj.id_prop),
                id_autor=autor["uri"].split("/")[-1],
                tipo_autor=autor["tipo"],
                ordem_assinatura=autor["ordemAssinatura"],
                proponente=autor["proponente"] == 1,
                praca='cd'
            )
            resp.append(autor_obj)
        autores_relacionadas += resp
    return autores_relacionadas

def fetch_proposicoes_detalhes(proposicoes: list[Proposicao]) -> list[ProposicaoDetalhes]:
    objs = []

    for proposicao in tqdm(proposicoes, desc="Baixando Proposições"):
        config = Config(endpoint=f'proposicoes/{proposicao.id_prop}')
        resp = get_data(config.url)[0]
        obj = ProposicaoDetalhes(
            id_prop = str(resp["id"]),
            sigla_tipo = resp["siglaTipo"],
            numero = resp["numero"],
            ano = resp["ano"],
            is_prl = True if resp["siglaTipo"]=="PRL" else False, 
            praca = "cd",
            data_apresentacao = resp["dataApresentacao"],
            ementa = resp["ementa"],
            cod_tipo = resp["codTipo"],
            tipo_descr = resp["descricaoTipo"],
            ementa_detalhada = resp["ementaDetalhada"],
            keywords = resp["keywords"],
            url_inteiro_teor = resp["urlInteiroTeor"],
            uri = resp["uri"],
            texto = resp["texto"],
            justificativa = resp["justificativa"],
            status_data = resp["statusProposicao"]["dataHora"],
            status_sequencia = resp["statusProposicao"]["sequencia"],
            status_sigla_orgao = resp["statusProposicao"]["siglaOrgao"],
            status_regime = resp["statusProposicao"]["regime"],
            status_descr_situacao = resp["statusProposicao"]["descricaoSituacao"],
            status_cod_tipo_situacao = resp["statusProposicao"]["codTipoTramitacao"],
            status_cod_situacao = resp["statusProposicao"]["codSituacao"],
            status_despacho = resp["statusProposicao"]["despacho"],
            url = resp["statusProposicao"]["url"],
            status_ambito = resp["statusProposicao"]["ambito"],
            status_apreciacao = resp["statusProposicao"]["apreciacao"],
            is_tramit = None,
            status_id_org = resp["statusProposicao"]['uriOrgao'].split('/')[-1]
        )

        objs.append(obj)
    
    return objs

In [5]:
projs = pd.read_excel(r'./data/projetos_selecionados.xlsx')

In [6]:
projs

,tipo,código,ano,tema,ementa,orientação
0,PEC,6,2019,reforma da previdência,"Modifica o sistema de previdência social, esta...",1
1,PL,3418,2021,fundebe,"Altera a Lei nº 14.113, de 25 de dezembro de 2...",1
2,PL,5829,2019,energia renovável,NOVA EMENTA: Institui o marco legal da microge...,1
3,PL,3179,2012,homeschooling,"Altera as Leis nºs 9.394, de 20 de dezembro de...",1


# Dados

In [7]:
df_dim_prof = pd.read_excel('./data/dim_profissoes_ficha.xlsx')
df_parls_profs = pd.read_csv('./data/fat_profissoes_2019-2023.csv')
df_profs_d = df_parls_profs[['id_parl', 'codTipoProfissao', 'titulo']].merge(df_dim_prof[['codTipoProfissao', 'is_empresario']], on='codTipoProfissao')
df_profs_d = df_profs_d.join(pd.get_dummies(df_profs_d['codTipoProfissao'])).groupby('id_parl').max(numeric_only=True).reset_index()

df_profs_d['id_parl'] = [str(id_parl) for id_parl in df_profs_d['id_parl']]

# calcular ativismos por tema

In [ ]:
for idx in projs.index:
    prop = Proposicao(
        sigla_tipo = projs.loc[idx, 'tipo'],
        numero = projs.loc[idx, 'código'],
        ano = projs.loc[idx, 'ano '],
    )

    resp = find_projects_by_name([prop])

    arvore = get_arvore_apensados(resp)

    autores = get_autores(arvore.nodes)

    props_detalhes = fetch_proposicoes_detalhes(arvore.nodes)

    df_props = pd.DataFrame(props_detalhes)
    df_autores = pd.DataFrame(autores)

    df_merge = df_autores.merge(df_props, on='id_prop', how='left').merge(df_profs_d, left_on='id_autor', right_on='id_parl', how='left')

    df_group = df_merge.groupby('id_autor').agg({'is_empresario': 'max', 'id_prop': 'count'})

    df_plot = pd.DataFrame()
    df_plot['Não Empresários'] = [g for g in df_group[df_group['is_empresario'] == 0]['id_prop']]
    df_ne =pd.DataFrame({'Empresários': [g for g in df_group[df_group['is_empresario'] == 1]['id_prop']]})

    df_plot=pd.concat([df_plot, df_ne], ignore_index=False, axis=1)

    print(projs.loc[idx, 'tema '])
    display(df_plot.agg(['min', 'max', 'mean', 'std', 'count']).round(decimals=2))

    fig, ax = plt.subplots()
    df_plot.plot.hist(ax=ax, alpha=0.8, bins=5)
    ax.set_ylabel('Freq.')
    ax.grid(axis='y')

In [12]:
def get_votacoes(list_props: list[Proposicao]) -> list[Votacao]:
    """get votacoes por proposicao"""
    votacoes = []
    id_proposicoes_com_votacoes = []
    for prop in tqdm(list_props, desc="Votacoes"):
        url = f"https://dadosabertos.camara.leg.br/api/v2/proposicoes/{prop.id_prop}/votacoes?ordem=DESC&ordenarPor=dataHoraRegistro"
        resp = request(url)
        if len(resp) == 0:
            continue
        id_proposicoes_com_votacoes.append(prop.id_prop)
        for vot in resp:
            votacoes.append(
                Votacao(
                    praca="cd",
                    id_votacao=vot["id"],
                    id_prop=prop.id_prop,
                    data_hora_registro=vot["dataHoraRegistro"],
                    sigla_orgao=vot["siglaOrgao"],
                    descricao=vot["descricao"],
                    aprovacao=vot["aprovacao"] == 1,
                )
            )
    # get detalhes das votações
    for votacao in tqdm(votacoes, desc="Detalhes Votações"):
        url = f"https://dadosabertos.camara.leg.br/api/v2/votacoes/{votacao.id_votacao}"
        resp = request(url)[0]
        votacao.uriProposicaoCitada = resp[
            "ultimaApresentacaoProposicao"
        ]["uriProposicaoCitada"]
        votacao.ultimaApresentacaoProposicaoDescricao = resp[
            "ultimaApresentacaoProposicao"
        ]["descricao"]
    print(
        f"{len(votacoes)} votações em {len(id_proposicoes_com_votacoes)} proposições."
    )
    return votacoes

In [17]:
votacoes = []
id_proposicoes_com_votacoes = []
for prop in tqdm(arvore.nodes, desc="Votações"):
    url = f"https://dadosabertos.camara.leg.br/api/v2/proposicoes/{prop.id_prop}/votacoes?ordem=DESC&ordenarPor=dataHoraRegistro"
    resp = request(url)
    if len(resp) == 0:
        continue
    id_proposicoes_com_votacoes.append(prop.id_prop)
    for vot in resp:
        votacoes.append(
            Votacao(
                praca="cd",
                id_votacao=vot["id"],
                id_prop=prop.id_prop,
                data_hora_registro=vot["dataHoraRegistro"],
                sigla_orgao=vot["siglaOrgao"],
                descricao=vot["descricao"],
                aprovacao=vot["aprovacao"] == 1,
            )
        )

Votacoes: 100%|██████████| 553/553 [07:21<00:00,  1.25it/s]


In [22]:
def get_votos(votacoes: [Votacao]):
    id_votacoes_com_votos = []
    votos = []
    for votacao in votacoes:
        url = f"https://dadosabertos.camara.leg.br/api/v2/votacoes/{votacao.id_votacao}/votos"
        resp = request(url)
        if len(resp) == 0:
            continue
        for vot in resp:
            votos.append(
                Voto(
                    praca="cd",
                    voto=vot["tipoVoto"],
                    id_parl=str(vot["deputado_"]["id"]),
                    sigla_partido=vot["deputado_"]["siglaPartido"],
                    id_votacao=votacao.id_votacao,
                )
            )
        id_votacoes_com_votos.append(votacao.id_votacao)
    print(f"{len(votos)} votos em {len(id_votacoes_com_votos)} votações.")
    return votos

'https://dadosabertos.camara.leg.br/api/v2/votacoes/{votacao.id_votacao}'

In [25]:
for votacao in tqdm(votacoes, desc="Detalhes Votações"):
    url = f"https://dadosabertos.camara.leg.br/api/v2/votacoes/{votacao.id_votacao}"
    resp = request(url)[0]
    votacao.uriProposicaoCitada = resp[
        "ultimaApresentacaoProposicao"
    ]["uriProposicaoCitada"]
    votacao.ultimaApresentacaoProposicaoDescricao = resp[
        "ultimaApresentacaoProposicao"
    ]["descricao"]
print(
    f"{len(votacoes)} votações em {len(id_proposicoes_com_votacoes)} proposições."
)

Detalhes Votações: 100%|██████████| 383/383 [01:54<00:00,  3.35it/s]

383 votações em 156 proposições.


In [32]:
id_votacoes_com_votos = []
votos = []
for votacao in tqdm(votacoes, "Votacoes analisadas"):
    url = f"https://dadosabertos.camara.leg.br/api/v2/votacoes/{votacao.id_votacao}/votos"
    resp = request(url)
    if len(resp) == 0:
        continue
    for vot in resp:
        votos.append(
            Voto(
                praca="cd",
                voto=vot["tipoVoto"],
                id_parl=str(vot["deputado_"]["id"]),
                sigla_partido=vot["deputado_"]["siglaPartido"],
                id_votacao=votacao.id_votacao,
            )
        )
    id_votacoes_com_votos.append(votacao.id_votacao)
print(f"{len(votos)} votos em {len(id_votacoes_com_votos)} votações.")

Votacoes analisadas: 100%|██████████| 383/383 [02:05<00:00,  3.05it/s]

19509 votos em 46 votações.


In [37]:
autores = get_autores(arvore.nodes)

Autores:  14%|█▍        | 78/553 [00:22<06:46,  1.17it/s]

In [ ]:
props_detalhes = fetch_proposicoes_detalhes(arvore.nodes)

In [35]:
df_props = pd.DataFrame(props_detalhes)
df_autores = pd.DataFrame(autores)


NameError: name 'autores' is not defined

# PLOT